<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



<h1>Curso Procesamiento de Lenguaje Natural</h1>

<h3>NER con LSTMs</h3>


<p> Julio Waissman Vilanova </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="150">
</p>


<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/pln/blob/main/labs/RNN/ner-lstm.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;"  width="30" /> Ejecuta en Colab</a>

Tomado parcialmente y adaptado de [el repositorio de github](https://github.com/Tekraj15/Named-Entity-Recognition-Using-LSTM-Keras/tree/master) 

</center>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

np.random.seed(0)
plt.style.use("ggplot")

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

print('Tensorflow version:', tf.__version__)
print('GPU detected:', tf.config.list_physical_devices('GPU'))

## Importando datos para NER

Para el NER, vanos a descargar un conjunto de datos de entrenamiento y prueba en español de la [Universidad de Antwerp](https://www.uantwerpen.be/en/), los cuales los puedes [consultar aquí](https://www.clips.uantwerpen.be/conll2002/ner/).

Y pues lo más fácil es descargarlos usando linea de comando:

In [ ]:
!rm esp.*
!curl -O https://www.clips.uantwerpen.be/conll2002/ner/data/esp.train.gz
!gunzip -f esp.train.gz 
!curl -O https://www.clips.uantwerpen.be/conll2002/ner/data/esp.testa.gz
!gunzip -f esp.testa.gz 
!curl -O https://www.clips.uantwerpen.be/conll2002/ner/data/esp.testb.gz
!gunzip -f esp.testb.gz 

Puedes abrir un archivo para ver que son palabras, así como su etiqueta usando el método de marcado comocido como [*BIO*](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)).

Vamos a leer un archivo y convertirlo a un `dataframe` de pandas para un manejo más fácil de la información. 

In [ ]:
def carga_dataframe(nombre):

  palabra = []
  tag = []
  sentencia = []

  with open(nombre, encoding='latin1') as fp:
    s = 1
    for line in fp.readlines():
      if len(line.strip()) == 0:
        s += 1
      else:
        palabra.append(line.split()[0].strip())
        tag.append(line.split()[1].strip())
        sentencia.append(s)

  df = pd.DataFrame({
    'Sentencia': sentencia,
    'Palabra': palabra,
    'Tag': tag
  })
  return df

Y la usamos para abrir el conjunto de datos de prueba y de aprendizaje:

In [ ]:
train_raw = carga_dataframe("esp.train")
test_raw = carga_dataframe("esp.testb")

print(f"Palabras únicas en el texto: {train_raw.Palabra.nunique()}")
print(f"Sentencias: {train_raw.Sentencia.max()}")
print(f"Etiquetas diferentes: {train_raw.Tag.nunique()}")

In [ ]:
train_raw.head()

Las etiquetas que tenemos son las siguientes:

In [ ]:
print(train_raw.Tag.value_counts())

## Preprocesando la información 

Para el preprocesamiento, vamos a encontrar los índices de cada uno de las palabras de nuestro vocabulario, así como los índices de cada una de las etiquetas. A las palabras se le agregan dos más, la palabra para un token desconocido, y la que indica que se acabó la frase.

In [ ]:
palabras = list(set(train_raw.Palabra.values))
palabras.append("ENDPAD")
palabras.append("UNK")
word2idx = {w: i + 1 for i, w in enumerate(palabras)}
num_palabras = len(palabras)

tags = list(set(train_raw.Tag.values))
tag2idx = {t: i for i, t in enumerate(tags)}
num_tags = len(tags)

print(num_palabras, num_tags)

Es interesante ver que al rededor de la cuarta parte de las palabras usadas en el conjunto de prueba, no se tienen en el conjunto de entrenamiento, por lo que este método va a ser muy limitado y es necesario extenderlo, usando vectores de palabras preentrenados.

In [ ]:
palabrasOOV = set(test_raw.Palabra.values) - set(train_raw.Palabra.values)
len(palabrasOOV)/ len(set(test_raw.Palabra.values))

Ya con los diccionarios de palabras a índices y de tags a índices, podemos entonces convertir nuestro dataframe en una lista, donde cada entrada sea una frase completa de pares ordenados.

In [ ]:
def agg_fun(df):
  return [
      (p, t) 
      for (p, t) in zip(
                        df.Palabra.values.tolist(),
                        df.Tag.values.tolist()
                       )
  ]

train_agg = train_raw.groupby('Sentencia').apply(agg_fun)
test_agg = test_raw.groupby('Sentencia').apply(agg_fun)


train_ls = [s for s in train_agg]
test_ls = [s for s in test_agg]

Veamos lo que sale:

In [ ]:
train_ls[0]

¿De que tamaño son cada una de las sentencias? Veamos en un histograma

In [ ]:
plt.hist([len(s) for s in train_ls], bins=50)
plt.title('Distribución de tamaño de las sentencias')
plt.xlabel('Tamaño en tokens')
plt.ylabel('Count')
plt.show()

Como vemos en el histograma, hay una muy pocas sentencias muy largas

In [ ]:
[len(s) for s in train_ls if len(s) > 150]

Así que vamos a recortarlas todas a un máximo de 150 para ver un histograma más claro:

In [ ]:
train_ls = [s if len(s) < 150 else s[:150] for s in train_ls]

plt.hist([len(s) for s in train_ls], bins=50)
plt.title('Distribución de tamaño de las sentencias')
plt.xlabel('Tamaño en tokens')
plt.ylabel('Count')
plt.show()

Lo que nos lleva a pensar que con sentencias de unos 80 tokens como máximo cubrimos casi todas las sentencias.


Ahora si, vamos a desarrollar unas secuencias de tokens de entrada y de datos de salida para hacer nuestro entrenamiento. Por cada sentencia, vamos a generar una secuencia de tokens de entrada y otra de salida.

max_len = 80

# Convertimos las palabras a tokens
X_idx = [[word2idx[w[0]] for w in s] for s in train_ls]
# Generamos las secuencias de entrada
X = keras.preprocessing.sequence.pad_sequences(
      maxlen=max_len, 
      sequences=X_idx, 
      padding="post", 
      value=num_palabras-1
)

# Convertimos los tags a tokens
y_idx = [[tag2idx[w[1]] for w in s] for s in train_ls]
# Generamos la secuencia de salidas
y = keras.preprocessing.sequence.pad_sequences(
      maxlen=max_len, 
      sequences=y_idx, 
      padding="post", 
      value=tag2idx["O"]
)

Y por últomo separamos nuestras secuencias en datos de entrenamiento y validación con `sklearn` como de costumbre 

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=1
)

# Modelo y entrenamiento

El modelo es muy sencillo y podemos discutirlo mucho y modificarlo para probar nuevas cosas.

Veamos el modelo, el cual lo vamos a definir en forma funcional:

In [ ]:
inputs = keras.Input(shape=(max_len,))

x = layers.Embedding(
    input_dim=num_palabras, 
    output_dim=64, 
    input_length=max_len
)(inputs)

x = layers.SpatialDropout1D(0.1)(x)

x = layers.Bidirectional(
      layers.LSTM(units=126, return_sequences=True, recurrent_dropout=0.1)
)(x)

outputs = layers.TimeDistributed(
      layers.Dense(num_tags, activation="softmax")
)(x)

model = keras.Model(inputs, outputs)

model.summary()

Y lo compilamos para que esté listo para el entrenamiento. Recuerda que si la salida del modelo es una *softmax* del tamaño de las salidas, pero la salida que tienes es simplemente el indice que debe ser (sin haber hecho *one hot encoding*) entonces tienes que seleccionar como función de pérdida `sparse_categorical_crossentropy`.

In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

Y para darle bien seguimiento vamos a establecer dos *checkpoints*, uno para ir guardand o los resultados intermedios, y otro para la detención prematura. Ninguno de los dos se necesita en este caso particular, pero no importa, vamos usándolo así por completud.

In [ ]:
chkpt = keras.callbacks.ModelCheckpoint(
    "model_weights.h5", 
    monitor='val_loss',
    verbose=1, 
    save_best_only=True, 
    save_weights_only=True, 
    mode='min'
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    min_delta=0, 
    patience=1, 
    verbose=0, 
    mode='max', 
    baseline=None, 
    restore_best_weights=False
)

Y ahora si, a poner a aprender este modelo:

In [ ]:
%%time

history = model.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_val,y_val),
    batch_size=32, 
    epochs=3,
    callbacks=[chkpt, early_stopping],
    verbose=1
)

## Verificación del modelo

Vamos primero graficando la pérdida y la métrica que decidimos mantener a partir de la información histórica del entrenamiento:

In [ ]:
plt.plot(history.history['accuracy'], '-o')
plt.plot(history.history['val_accuracy'], '-o')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['entrenamiento', 'validación'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'], '-o')
plt.plot(history.history['val_loss'], '-o')
plt.title('Pérdida durante el aprendizaje')
plt.ylabel('pérdida')
plt.xlabel('epoch')
plt.legend(['entrenamiento', 'validación'], loc='upper right')
plt.show()

Y ahora vamos a ver como se comporta una sentencia en particular (la que sea). Así que vamos a simular con alguna secuencia que venga en el conjunto de validación:

In [ ]:
i = np.random.randint(0, x_val.shape[0]) 

x_i = np.array([x_val[i]])
y_true = y_val[i]

prediction = model.predict(x_i)
y_est = np.argmax(prediction, axis=-1)[0]


print("\n" + "Palabra".ljust(15) + "Real".ljust(10) + "Estimada" )
print("\n" + "-" * 40)

for w, true, pred in zip(x_i[0], y_true, y_est):
  print( palabras[w-1].ljust(15) + 
         tags[true].ljust(10) + 
         tags[pred]
  )

Ahora vamos a hacer la verificación mas dificil de hacer, la que incluye a otro conjunto de prueba. Para esto tenemos que procesar el conjunto de prueba que ya tenemos bien identificado desde el inicio:

Xt_idx = [[word2idx.get(w[0], num_palabras - 1) for w in s] for s in test_ls]
Xt = keras.preprocessing.sequence.pad_sequences(
      maxlen=max_len, 
      sequences=Xt_idx, 
      padding="post", 
      value=num_palabras-1
)

yt_idx = [[tag2idx[w[1]] for w in s] for s in test_ls]
yt = keras.preprocessing.sequence.pad_sequences(
      maxlen=max_len, 
      sequences=yt_idx, 
      padding="post", 
      value=tag2idx["O"]
)

Y evaluamos el modelo con nuevos datos nunca vistos para el algorítmo:

In [ ]:
model.evaluate(Xt,yt)

Recuerda que estos datos tienen muchos valores que no conocia el método (los tokens tipo OOV)

In [ ]:
i = np.random.randint(0, Xt.shape[0]) 

x_i = np.array([Xt[i]])
y_true = yt[i]

prediction = model.predict(x_i)
y_est = np.argmax(prediction, axis=-1)[0]


print("\n" + "Palabra".ljust(15) + "Real".ljust(10) + "Estimada" )
print("\n" + "-" * 40)

for w, true, pred in zip(x_i[0], y_true, Y_est):
  print( palabras[w-1].ljust(15) + 
         tags[true].ljust(10) + 
         tags[pred]
  )

## Tarea

Vamos resolviendo el problema de los OOV, cambiando el uso de un vector de *embeddings* que entrenamos a usar directamente un vector preentrenado y bien conocido, usando [*GLOVe*](https://nlp.stanford.edu/projects/glove/), [*Word2Vec*](https://www.tensorflow.org/text/tutorials/word2vec) o [*FastText*](https://fasttext.cc) entre otros.

Varios *embeddings* se pueden encontrar [en este repositorio de GitHub](https://github.com/dccuchile/spanish-word-embeddings). Aunque para *FastText* te recomiendo usar la [librería propia](https://fasttext.cc/docs/en/unsupervised-tutorial.html), así como el [modelo preentrenado oficial en español](https://fasttext.cc/docs/en/crawl-vectors.html). *Glove* y *Word2vec* se pueden usar sin problema como dccionarios, o de forma eficiente con la librería [Gensim](https://radimrehurek.com/gensim/index.html)